In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.convergence import convergence_in_time, convergence_in_space
from MPyDATA_examples.Olesik_et_al_2020.distributions import n_n, n_s
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_ln, x_log_of_p3
from MPyDATA_examples.utils.error_norms import L2
from MPyDATA_examples.utils.show_plot import show_plot

import matplotlib.pyplot as plt
import numpy as np

In [3]:
opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
)

In [4]:
plot_setup = {"psi_coord":x_p2(), "grid_layouts":(x_id(), x_p2(), x_ln()), "dt":.33,"nr":75}

In [5]:
def convergence_in_space(nrs: list, opt_set, plot_setup):
    err_data = {}
    for grid_layout in plot_setup['grid_layouts']:
        grid_layout_str = grid_layout.__class__.__name__
        err_data[grid_layout_str] = {}
        for opts in opt_set:
            err_data[grid_layout_str][str(opts)] = {'dx':[], 'error_L2':[]} 

    for nri, nr in enumerate(nrs):
        data, setup = compute_figure_data(opt_set = opt_set, **{**plot_setup, **{'nr': nr}})
        for grid_layout in data.keys():
            for opts in data[grid_layout]["error_L2"].keys():
                err_data[grid_layout][str(opts)]['dx'].append(data[grid_layout]["grid"]['dx'])
                err_data[grid_layout][str(opts)]['error_L2'].append(data[grid_layout]["error_L2"][opts])
    return err_data

In [6]:
nrs = [32, 64] 
err_data_nr = convergence_in_space(nrs, opt_set, plot_setup)

In [7]:
for grid_layout in plot_setup['grid_layouts']:
    grid_layout_str = grid_layout.__class__.__name__
    for i, opts in enumerate(err_data_nr[grid_layout_str].keys()):
        x = np.log2(err_data_nr[grid_layout_str][opts]['dx'])
        y = err_data_nr[grid_layout_str][opts]['error_L2']
        plt.plot(x, y, label=opts, linestyle=':')
        plt.scatter(x, y)
    plt.title(f"grid layout: {grid_layout_str}")
    
    for k in [2,3]:
        plt.plot(x, k*x+0, '--', label=f"error ∝ $Δx^{k}$", color='black')
        
    plt.legend()
    plt.gca().set_xlabel('log$_2$(dx)')
    plt.gca().set_ylabel('log$_2(err_{L2})$')
    plt.grid()
    show_plot()

In [8]:
def convergence_in_time(dts: list, opt_set, plot_setup):
    err_data = {}
    for grid_layout in plot_setup['grid_layouts']:
        grid_layout_str = grid_layout.__class__.__name__
        err_data[grid_layout_str] = {}
        for opts in opt_set:
            err_data[grid_layout_str][str(opts)] = {'dx':[], 'error_L2':[]} 

    for dti, dt in enumerate(dts):
        data, setup = compute_figure_data(opt_set = opt_set, **{**plot_setup, **{'dt': dt}})
        for grid_layout_str in data.keys():
            for opts in data[grid_layout_str]["error_L2"].keys():
                err_data[grid_layout_str][str(opts)]['dx'].append(data[grid_layout_str]["grid"]['dx'])
                err_data[grid_layout_str][str(opts)]['error_L2'].append(data[grid_layout_str]["error_L2"][opts])
    return err_data

In [ ]:
dts = [.25, .125]
err_data_dt = convergence_in_time(dts, opt_set, plot_setup)

In [ ]:
x = np.log2(dts)
for coord in results.keys():
    for i, opts in enumerate(err_data_dt[coord].keys()):
        plt.plot(x, err_data_dt[coord][opts], label=opts, linestyle=':')
        plt.scatter(x, err_data_dt[coord][opts])
    plt.title(coord)
    plt.legend()
    
    for k in [2,3]:
        plt.plot(x, k*x-0.5, ':', label=f"error ∝ $Δx^{k}$", color='black')
    
    plt.gca().set_xlabel('log$_2$(dt)')
    plt.gca().set_ylabel('log$_2(err_{L2})$')
    plt.grid()

    show_plot()